In [2]:
import nltk
from nltk.corpus import brown
from nltk.tokenize import sent_tokenize
import string
from collections import defaultdict
from collections import Counter
import numpy as np
import math

In [3]:
corp = brown.sents(categories='news')
brown_news_tagged = brown.tagged_sents(categories='news', tagset='universal')

In [4]:
flat_corp = [item for l in corp for item in l]

In [5]:
flat_brown = [item for l in brown_news_tagged for item in l]

In [6]:
pi_DET = [1]

In [7]:
b = Counter(flat_brown)

In [29]:
b_ik = defaultdict(dict)
for w_pos, c in b.items():
    b_ik[w_pos[0]][w_pos[1]] = c

In [9]:
denom = defaultdict(int)
for w_pos, c in b.items():
    denom[w_pos[1]] += c

In [10]:
# skip this
for w, d in b_ik.items():
    for pos, c in d.items():
        d[pos] = c * 1.0 / denom[pos]

In [37]:
a_ij = defaultdict(dict)

for c in range(0, len(flat_brown) - 1):
    if flat_brown[c+1][1] in a_ij[flat_brown[c][1]]:
        a_ij[flat_brown[c][1]][flat_brown[c+1][1]] += 1
    else:
        a_ij[flat_brown[c][1]][flat_brown[c+1][1]] = 1

In [12]:
# skip this
for pos, d in a_ij.items():
    for p, c in d.items():
        d[p] = c * 1.0 / denom[pos]

In [13]:
# checking if rows in a_ij add up to 1
for pos, d in a_ij.items():
    count = 0
    for p, c in d.items():
        count += c
    print(pos + ": "+ str(count))

ADV: 1.0000000000000002
PRON: 1.0
NOUN: 1.0
CONJ: 0.9999999999999999
ADJ: 1.0
PRT: 0.9999999999999999
NUM: 1.0
X: 1.0
.: 0.9999161636485581
VERB: 0.9999999999999999
DET: 1.0000000000000002
ADP: 1.0


In [14]:
# word to number dictionary
o_index = dict(zip(b_ik, range(len(b_ik))))

In [30]:
# number to word dictionary
o_index_n = dict(zip(range(len(b_ik)), b_ik))

In [15]:
# POS to number dictionary
s_index = dict(zip(a_ij, range(len(a_ij))))

In [32]:
# number to POS dictionary
s_index_n = dict(zip(range(len(a_ij)), a_ij))

In [16]:
em = np.zeros((len(s_index), len(o_index)))
tm = np.zeros((len(s_index), len(s_index)))

In [17]:
for o, s in b_ik.items():
    for pos, prob in s.items():
        em[s_index[pos]][o_index[o]] = prob

In [31]:
b_ik

defaultdict(dict,
            {'205-pound': {'ADJ': 1},
             'man': {'NOUN': 72},
             'Famed': {'ADJ': 1},
             'belted': {'VERB': 1},
             'grown': {'VERB': 1},
             'Sioux': {'NOUN': 1},
             'Could': {'VERB': 1},
             'offensive': {'ADJ': 1},
             'apprentices': {'NOUN': 1},
             'low-down': {'ADJ': 1},
             'undertaken': {'VERB': 2},
             'athlete': {'NOUN': 2},
             'Kelly': {'NOUN': 1},
             'workshop': {'NOUN': 5},
             'bespectacled': {'ADJ': 1},
             '16-year-old': {'ADJ': 1},
             'Aikin': {'NOUN': 1},
             'delegations': {'NOUN': 1},
             'clients': {'NOUN': 2},
             'seconds': {'NOUN': 2},
             'romp': {'NOUN': 1},
             'Ivory': {'NOUN': 1},
             'unit': {'NOUN': 10},
             'Central': {'ADJ': 15},
             'quipping': {'VERB': 1},
             'Cezannes': {'NOUN': 1},
             'Hewlett

In [35]:
# Laplace smoothing for emission matrix

for col in range(0, len(em[0])):
    for row in range(0, len(em)):
        pos_list = b_ik[o_index_n[col]]
        if (s_index_n[row] in pos_list):
            em[row][col] = (b_ik[o_index_n[col]][s_index_n[row]] * 1.0 + 1) / (denom[s_index_n[row]] + len(em))
        else:
            em[row][col] = (1.0) / (denom[s_index_n[row]] + len(em))

In [36]:
em

array([[  2.97530497e-04,   2.97530497e-04,   2.97530497e-04, ...,
          2.97530497e-04,   5.95060994e-04,   2.97530497e-04],
       [  3.92618767e-04,   3.92618767e-04,   3.92618767e-04, ...,
          3.92618767e-04,   3.92618767e-04,   3.92618767e-04],
       [  3.26094046e-05,   2.38048653e-03,   3.26094046e-05, ...,
          6.52188091e-05,   3.26094046e-05,   3.26094046e-05],
       ..., 
       [  6.93914371e-05,   6.93914371e-05,   6.93914371e-05, ...,
          6.93914371e-05,   6.93914371e-05,   4.85740060e-04],
       [  8.77116042e-05,   8.77116042e-05,   8.77116042e-05, ...,
          8.77116042e-05,   8.77116042e-05,   8.77116042e-05],
       [  8.08603542e-05,   8.08603542e-05,   8.08603542e-05, ...,
          8.08603542e-05,   8.08603542e-05,   8.08603542e-05]])

In [41]:
for row in range(0, len(tm)):
    for col in range(0, len(tm[0])):
        pos_list = a_ij[s_index_n[row]]
        if (s_index_n[col] in pos_list):
            tm[row][col] = (a_ij[s_index_n[row]][s_index_n[col]] * 1.0 + 1) / (denom[s_index_n[row]] + len(tm))
        else:
            tm[row][col] = (1.0) / (denom[s_index_n[row]] + len(em))

In [42]:
tm

array([[  7.55727462e-02,   3.65962511e-02,   5.56382029e-02,
          1.60666468e-02,   1.20202321e-01,   2.85629277e-02,
          2.43975007e-02,   2.97530497e-04,   1.32698602e-01,
          2.72835466e-01,   8.12258256e-02,   1.55905980e-01],
       [  5.85001963e-02,   7.06713781e-03,   7.85237534e-03,
          1.21711818e-02,   9.42285041e-03,   2.19866510e-02,
          1.57047507e-03,   3.92618767e-04,   6.43894778e-02,
          7.58146839e-01,   1.25638005e-02,   4.59363958e-02],
       [  2.06743625e-02,   1.22285267e-02,   2.59570860e-01,
          4.75445118e-02,   1.70547186e-02,   1.68590622e-02,
          1.05654471e-02,   3.58703450e-04,   2.52168525e-01,
          1.36731233e-01,   1.36307311e-02,   2.12613318e-01],
       [  5.82631000e-02,   4.25064126e-02,   3.44082081e-01,
          3.66434591e-04,   1.07731770e-01,   2.45511176e-02,
          2.78490289e-02,   7.32869183e-04,   1.72224258e-02,
          1.75888604e-01,   1.43275925e-01,   5.75302309e-02],
    

In [18]:
# skip this
for si, pos in a_ij.items():
    for sj, prob in pos. items():
        tm[s_index[si]][s_index[sj]] = prob

In [43]:
def helper(col, tm):
    mx = (-1, -math.inf)

    for row in range(len(col)):
        value = col[row]
        value += math.log(tm[row])

        if value > mx[1]:
            mx = (row, value)

    return mx

In [44]:
def run(em, tm, obs, init):
    n = len(em); T = len(obs)
    # step 1: init n x T matrix
    ls = np.zeros((n, T))
    phi = np.zeros((n, T))


    # col 1 in ls -> log(pi_i*bi_o1)
    for r in range(len(init)):
        value = init[r]*em[r][o_index[obs[0]]]
        ls[r][0] = math.log(value)


    #step 2: fill in tables by col
    for t in range(1,T): # t -> col (time)

        for s in range(0,n): # s -> row (state)
            prob_event = math.log( em[s][o_index[obs[t]]] ) # log( bj(Ot+1))

            mx = helper(ls[:, t-1], tm[s][:])

            value = prob_event + mx[1]
            ls[s][t] = value
            phi[s][t] = mx[0]

    last_col = ls[:, np.shape(ls)[1]-1]

    start = np.argmax(last_col)
    path = [start]

    for time in range(T-1, 0, -1):
        start = phi[int(start)][time]
        path = np.insert(path, 0, int(start))
    
    return path

In [45]:
init = np.zeros(len(s_index))
init[s_index['DET']] = 1

In [46]:
path = run(em, tm, flat_corp, init)

ValueError: math domain error

[7 1 7 1 7 1 7 1 7 1 7 1 7 1 7 1 7 1 7 1]


In [24]:
flat_corp[-100:]

['Office',
 'of',
 'Education',
 '.',
 'And',
 'over',
 '66',
 'per',
 'cent',
 'of',
 'the',
 'elementary',
 'schools',
 'with',
 '150',
 'or',
 'more',
 'pupils',
 'do',
 'not',
 'have',
 'any',
 'library',
 'at',
 'all',
 '.',
 'In',
 'every',
 'aspect',
 'of',
 'service',
 '--',
 'to',
 'the',
 'public',
 ',',
 'to',
 'children',
 'in',
 'schools',
 ',',
 'to',
 'colleges',
 'and',
 'universities',
 '--',
 'the',
 'library',
 'of',
 'today',
 'is',
 'failing',
 'to',
 'render',
 'vitally',
 'needed',
 'services',
 '.',
 'Only',
 'public',
 'understanding',
 'and',
 'support',
 'can',
 'provide',
 'that',
 'service',
 '.',
 'This',
 'is',
 'one',
 'of',
 'the',
 'main',
 'reasons',
 'for',
 'National',
 'Library',
 'Week',
 ',',
 'April',
 '16-22',
 ',',
 'and',
 'for',
 'its',
 'theme',
 ':',
 '``',
 'For',
 'a',
 'richer',
 ',',
 'fuller',
 'life',
 ',',
 'read',
 "''",
 '!',
 '!']

In [25]:
flat_corp[:10]

['The',
 'Fulton',
 'County',
 'Grand',
 'Jury',
 'said',
 'Friday',
 'an',
 'investigation',
 'of']

In [159]:
s_index

{'.': 7,
 'ADJ': 2,
 'ADP': 3,
 'ADV': 6,
 'CONJ': 10,
 'DET': 0,
 'NOUN': 5,
 'NUM': 8,
 'PRON': 1,
 'PRT': 4,
 'VERB': 11,
 'X': 9}

In [30]:
if __name__ == "__main__":
    main()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.']
4623
defaultdict(<class 'int'>, {'DET': 1116, 'PRON': 435, 'ADJ': 208, 'ADP': 456, 'PRT': 127, 'NOUN': 1208, 'ADV': 251, '.': 405, 'NUM': 77, 'X': 2, 'CONJ': 168, 'VERB': 170})
4623


In [29]:
def main():
    corp = brown.sents(categories='news')
    print(corp[0])
    print(corp[1])

    brown_news_tagged = brown.tagged_sents(categories='news', tagset='universal')

    pi_s, count = pi(brown_news_tagged)
    print(pi_s)
    print(count)

In [35]:
def pi(sents):
    pos = defaultdict(int)
    count = 0
    for s in sents:
        pos[s[0][1]] += 1
        count += 1
    return pos, count